# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка.
Необходимо выявить закономерность между семейным положением и благостостоянием заемщика на вероятность погашения кредита в установленный срок. Иными словами способна ли определенная семья расплатиться с банком в срок  

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

*Описание таблицы датафрейма:* 
-	children — количество детей в семье 
-	days_employed — общий трудовой стаж в днях 
-	dob_years — возраст клиента в годах 
-	education — уровень образования клиента 
-	education_id — идентификатор уровня образования 
-	family_status — семейное положение 
-	family_status_id — идентификатор семейного положения 
-	gender — пол клиента 
-	income_type — тип занятости 
-	debt — имел ли задолженность по возврату кредитов 
-	total_income — ежемесячный доход 
-	purpose — цель получения кредита 

*customer_solvency - с англ. 'платежеспособность клиентов'*


## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

customer_solvency = pd.read_csv('/datasets/data.csv') 

*Взглянем на датафрейм*

In [2]:
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


*Проверим названия колонок и выведем общую информацию о данных*

In [3]:
customer_solvency.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

In [4]:
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


*Проверим в каких колонках и какое количество пропусков*

In [5]:
customer_solvency.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

*Проверим количество дубликатов*

In [6]:
customer_solvency.duplicated().sum()

54

In [7]:
customer_solvency.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### **Вывод**

<div style="border:solid indigo 2px; padding: 20px">
    
- Датафрейм имеет 21525 строк, достаточно, чтобы сделать выводы (если там не все дубликаты конечно).
- Названия столбцов в порядке: латинскими строчными буквами, без пробелов, в "змеином_регистре". Переименование не требуется.
- Солбцы 'days_employed' и 'total_income' имеют пропуски, ровно по 2174 шт, нужно их обработать.
- Столбец days_employed имеет тип float64, лучше перевести в целочисленные значения (зачем нам тысячные доли дней?). Также столбец имеет отрицательные значения, нужно обратить внимание.
- Столбец 'total_income' (ежемесячный доход) можно привести к целочисленному значению, точность до 6 знаков здесь ни к чему.
- Датафрейм имеет 54 дубликата.

## Шаг 2. Предобработка данных

### Обработка пропусков

*Посмотрим где и сколько у нас пропусков*

In [8]:
customer_solvency.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

*сколько процентов ушло на пропуски в строках?*

In [9]:
print(f'Пропуски составляют {customer_solvency.shape[0] / 2174 / 100:.0%} от общего числа строк')

Пропуски составляют 10% от общего числа строк


##### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">

У нас 2174 пропусков NaN в двух столбцах. Это 10% данных.
Пропуски в столбце 'дни стажа' и 'ежемесячный доход'. \
**Вероятная причина в том, что если не заполнили колонку 'дней стажа', значит и заработка нет. И наоборот. Возможно данные клиенты являются самозанятыми и не устроены официально.**

Самый легкий способ устранения пропусков - просто удалить их. Но давайте не будем торопиться.



#### Про столбец 'days_employed'

- отрицательные значения дней стажа получились путем вычитания года начала работы (около 18 лет) от прожитых лет 
- огромные положительные значения взяты ошибочно. Вероятно, расчет производился по формуле:

                   ('кол-во лет' - 'год начала работы') * 365.
                   
                   Но потом, по ошибке, еще умножили на кол-во часов
                   ('кол-во лет' - 'год начала работы') * 365 * 24
                   
**Обработаем пропуски, заменив NaN по формуле ('кол-во лет' - '18') * 365**

In [10]:
def change_nan_days_employed(row):
    
    # функция меняет NaN на ('dob_years' - '18') * 365
    
    row['days_employed'] = row['days_employed'].fillna((row['dob_years'] - 18)*365)
    return row

In [11]:
customer_solvency = change_nan_days_employed(customer_solvency)
customer_solvency.isna().sum()

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Обработаем отрицательные дни стажа.**

In [12]:
customer_solvency['days_employed'] = abs(customer_solvency['days_employed'])
customer_solvency.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Обработаем большие положительные числа в 'days_employed'.** \
*Вероятно ошибка в формуле: домножили на кол-во часов. \
Максимальный стаж в днях подсчитаем по формуле: 
(65 (выход на пенсию) - 18 (начало работы) ) * 365 = 17155 дней \
      Все, что больше 17155 предлагаю разделить на 24*

In [13]:
def change_large_data(row):
    if row > (65-18)*365:
        row /= 24
    return row

In [14]:
customer_solvency['days_employed'] = customer_solvency['days_employed'].apply(change_large_data)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


#### Про столбец 'total_income'

**Будем менять пропуски на медианные значения ежемесячного дохода в зависимости от категории клиента 'income_type'**

*Выясним, какой медианный заработок у каждого типа занятости?*

In [15]:
customer_solvency \
                    .groupby(by='income_type')['total_income'] \
                    .median() \
                    .sort_values(ascending=False)     

income_type
предприниматель    499163.144947
компаньон          172357.950966
госслужащий        150447.935283
сотрудник          142594.396847
безработный        131339.751676
пенсионер          118514.486412
студент             98201.625314
в декрете           53829.130729
Name: total_income, dtype: float64

*Проверим в каких категориях есть пропуски данных.
Для этого заменим NaN на 0:*

In [16]:
customer_solvency['total_income'] = customer_solvency['total_income'].fillna(0)
print(customer_solvency['income_type'][customer_solvency['total_income'] == 0].value_counts())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64


*Проверим, избавились ли мы от пропусков в столбце 'total_income', заменив их на 0*

In [17]:
customer_solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Получается у 5ти категорий есть пропуски: 'сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель'**

*Создадим функцию для заполнения нулевых значений по условию.*

income_type = 'сотрудник', 'компаньон', 'пенсионер', 'госслужащий', 'предприниматель' -- список категорий, где есть пропуски \
income_avg = 142594.396847, 172357.950966, 118514.486412, 150447.935283, 499163.144947  -- список с медианным заработком

In [18]:
def fill_zero_total_income(row):
    
    # функция проверяет тип занятости и наличие нулевого значения дохода,
    # если условия дают истину, то заменяет на медианное значение дохода соответствующей категории занятости
    # если условия дают ложь, оставляет старое значение

    if row['income_type'] == 'сотрудник':
        if row['total_income'] == 0:
            new_value = 142594.396847
            return new_value
        return row['total_income']

    elif row['income_type'] == 'компаньон':
        if row['total_income'] == 0:
            new_value = 172357.950966
            return new_value
        return row['total_income']
    
    elif row['income_type'] == 'пенсионер':
        if row['total_income'] == 0:
            new_value = 118514.486412
            return new_value 
        return row['total_income']      
    
    elif row['income_type'] == 'госслужащий':
        if row['total_income'] == 0:
            new_value = 150447.935283
            return new_value 
        return row['total_income']  
    
    elif row['income_type'] == 'предприниматель':
        if row['total_income'] == 0:
            new_value = 499163.144947
            return new_value  
        return row['total_income']  

In [19]:
customer_solvency['total_income'] = customer_solvency.apply(fill_zero_total_income, axis=1)
customer_solvency.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [20]:
customer_solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        4
purpose             0
dtype: int64

*Появилось 4 пропуска в "total_income". Удалим эти 4 пропуска.*

In [21]:
customer_solvency = customer_solvency.dropna(subset={'total_income'}).reset_index(drop=True)
customer_solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

#### Про столбец 'children'

*Проверим данные столбца 'children'*

In [22]:
customer_solvency['children'].value_counts()

 0     14147
 1      4817
 2      2054
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

*значение -1. Вероятно это человеческая ошибка. Исправим на 1* \
*значение 20. Вероятно это человеческая ошибка. Исправим на 2*

In [23]:
customer_solvency['children'] = abs(customer_solvency['children'])

In [24]:
customer_solvency['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'предприниматель'], dtype=object)

In [25]:
customer_solvency.loc[customer_solvency['children'] == 20, 'children'] = 2
customer_solvency['children'].value_counts()

0    14147
1     4864
2     2130
3      330
4       41
5        9
Name: children, dtype: int64

#### **Вывод**

<div style="border:solid indigo 2px; padding: 20px">
    
- Пропуски в датафрейме найдены в двух столбцах: 'days_employed' и 'total_income'. Количество пропусков - почти 10%.
- Столбец 'days_employed':
    - Пропуски заменим на стаж, подсчитанный по формуле: (лет клиенту - 18)*365
    - Отрицательные значения дней стажа получились путем вычитания года начала работы (около 18 лет) от прожитых лет.
    - Огромные положительные значения взяты ошибочно. Вероятно расчет производился по формуле:

                   ('кол-во лет' - 'год начала работы') * 30 * 12. Но потом еще умножили на кол-во часов
                   ('кол-во лет' - 'год начала работы') * 30 * 12 * 24 , что является ошибкой

    
- В столбце 'total_income' принято решение не удалять пропуски, а заменить их на медианное значение ежемесячного дохода в зависимости от категории клиента. Замена выполнена функцией.
- В столбце 'children' значения -1 заменили на 1. Вероятно это была человеческая ошибка, такая же как 20 детей, которую заменили на 2.

### Замена типа данных

- *Столбец 'total_income' содержит вещественные значения. Такие данные не удобны, точность дохода до шести нулей не нужна. Необходимо перевести данные в целочисленные значения.* 
- *Также заменим количество дней стажа на integer*

In [26]:
customer_solvency.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21521 entries, 0 to 21520
Data columns (total 12 columns):
children            21521 non-null int64
days_employed       21521 non-null float64
dob_years           21521 non-null int64
education           21521 non-null object
education_id        21521 non-null int64
family_status       21521 non-null object
family_status_id    21521 non-null int64
gender              21521 non-null object
income_type         21521 non-null object
debt                21521 non-null int64
total_income        21521 non-null float64
purpose             21521 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [27]:
customer_solvency.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [28]:
customer_solvency['days_employed']

0         8437.673028
1         4024.803754
2         5623.422610
3         4124.747207
4        14177.753002
             ...     
21516     4529.316663
21517    14330.725172
21518     2113.346888
21519     3112.481705
21520     1984.507589
Name: days_employed, Length: 21521, dtype: float64

In [29]:
customer_solvency['total_income'] = customer_solvency['total_income'].astype('int')
customer_solvency['days_employed'] = customer_solvency['days_employed'].astype('int')

In [30]:
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


#### **Вывод**

<div style="border:solid indigo 2px; padding: 20px">
    
- Столбец 'total_income' содержит вещественные значения. Такие данные не удобны для просмотра, точноcть дохода до шести нулей не нужна. Перевели данные в целочисленные значения.
- Столбец 'days_employed' также перевели в целые значения, тк доли дней нам не нужны.
- Для перевода используем функцию astype()

### Обработка дубликатов

#### Обработка неявных дубликатов

*Дубликаты встречаются в столбце 'education'*

In [31]:
customer_solvency['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

*Значения отличаются только регистром. Предлагаю использовать варианты в нижнем регистре. Значит нужно весь столбец перевести к нижнему регистру методом lower()*

In [32]:
customer_solvency['education'] = customer_solvency['education'].str.lower()
customer_solvency['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

*Теперь неявных дубликатов в столбце 'education' нет*

#### Обработка явных дубликатов

*Выведем количество явных дубликатов*

In [33]:
customer_solvency.duplicated().sum()

71

*Посмотрим на явные дубликаты*

In [34]:
duplicated_data = customer_solvency[customer_solvency.duplicated()].head(20)
duplicated_data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,8395,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3289,0,14600,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4181,1,5840,34,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4850,0,15330,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5556,0,14600,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
6311,0,4380,30,среднее,1,женат / замужем,0,M,сотрудник,0,142594,строительство жилой недвижимости
7807,0,14235,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7920,0,16790,64,высшее,0,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
7937,0,806,71,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
8582,0,14600,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование


*Удалим явные дубликаты функцией drop_duplicates(), а также сбросим индексы, чтобы все шло по порядку reset_index(drop=True)*

In [35]:
customer_solvency = customer_solvency.drop_duplicates().reset_index(drop=True)

*Проверим еще раз наличие дубликатов*

In [36]:
customer_solvency.duplicated().sum()

0

#### **Вывод**

<div style="border:solid indigo 2px; padding: 20px">

- В столбце 'education' значения отличались только регистром. Такая ошибка возникла из-за человеческого фактора. Кто-то писал строчными, кто-то прописными. Неявные дубликаты были устранены путем приведения данных к нижнему регистру методом lower().
- В данных были выявлено 71 явных дубликата. Такие дубликаты возможно получились путем технической ошибки. Были удалены функцией drop_duplicates(), а также сброшены индексы, чтобы все шло по порядку reset_index(drop=True)

### Лемматизация

#### Вариант 1 

*Выведем уникальные значения столбца. Посмотрим, с чем придется работать*

In [37]:
data_for_lemmitize = customer_solvency['purpose'].unique()
data_for_lemmitize

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

*Имеем множество повторяющихся слов в разных падежах. То, что нужно для лемматизации.*

In [38]:
lemmas = m.lemmatize(str(data_for_lemmitize))
lemmas

["['",
 'покупка',
 ' ',
 'жилье',
 "' '",
 'приобретение',
 ' ',
 'автомобиль',
 "' '",
 'дополнительный',
 ' ',
 'образование',
 "'\n",
 " '",
 'сыграть',
 ' ',
 'свадьба',
 "' '",
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 "' '",
 'образование',
 "'\n",
 " '",
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 "' '",
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 "' '",
 'покупка',
 ' ',
 'недвижимость',
 "'\n",
 " '",
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 "' '",
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 "'\n",
 " '",
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 "' '",
 'недвижимость',
 "'\n",
 " '",
 'строительство',
 ' ',
 'недвижимость',
 "' '",
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 "'\n",
 " '",
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 "' '",
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 "'\n",
 " '",
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недви

*Выясним, на что берут кредит чаще всего*

In [39]:
print(Counter(lemmas))

Counter({' ': 59, "' '": 23, "'\n": 14, " '": 14, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, "['": 1, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, "']\n": 1})


 - После проведения лемматизации выяснили, на что чаще всего брали кредит: 
         1е место - недвижимость 
         2е место - покупка автомобиля 
         3е место - образование
         
**Но данный метод не показателен и с его результатами тяжело работать.**

#### Вариант 2

*Лемматизацию будем производить в функции, а потом создадим на основе нее новый столбец 'purpose_loan' (цели кредита)*

In [40]:
def category_loan(row):
    
 #  функция принимает датафрейм
 #  проводит лемматизацию столбца 'purpose'
 #  проверяет наличие слов в лемматизированной переменной
 #  возвращает соответствующее слово
    
    lem_purpose = m.lemmatize(str(row['purpose']))
    if 'автомобиль' in lem_purpose:
        return 'автомобиль'
    elif 'образование' in lem_purpose:
        return 'образование'  
    elif 'недвижимость' in lem_purpose or 'жилье' in lem_purpose:
        return 'кв. м.'    
    elif 'свадьба' in lem_purpose:
        return 'свадьба'          
    elif 'ремонт' in lem_purpose:
        return 'ремонт'      

*Создадим новый столбец с лемматизированными значениями, чтобы понимать причины взятия кредита*

In [41]:
customer_solvency['purpose_loan'] = customer_solvency.apply(category_loan, axis=1)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_loan
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,кв. м.
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,кв. м.
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


In [42]:
customer_solvency['purpose_loan'].value_counts()

кв. м.         10808
автомобиль      4305
образование     4013
свадьба         2324
Name: purpose_loan, dtype: int64

*Результаты совпадают с предыдущим вариантом.* В отличие от 1го варианта сейчас с данными удобно работать.

#### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">
    
 В столбце 'purpose'(цели кредита) большое количество слов в разных падежах. Применим лемматизацию чтобы вернуть леммы, базовые формы флективных слов.
    
 - После проведения лемматизации выяснили, на что чаще всего брали кредит: 
         1е место - для недвижимости 
         2е место - для автомобиля 
         3е место - для образования
         4е место - для свадьбы

### Категоризация данных

**Категоризировать данные будем согласно поставленным задачам**

#### для задачи №1 (Количество детей - задолженность)

**Создадим новую таблицу. Основа для таблицы столбы 'children' и 'debt'. \
Третий столбец будет использован для подсчета количества строк в сводной таблице**

In [43]:
children_debt = customer_solvency[['children','debt', 'purpose_loan']]

*На основе нее сделаем сводную таблицу*

In [44]:
children_debt_pivot = children_debt.pivot_table(index='children', columns='debt', values='purpose_loan', aggfunc='count')
children_debt_pivot

debt,0,1
children,,
0,13026.0,1063.0
1,4410.0,444.0
2,1926.0,201.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


#### для задачи №2 (Наличие детей - задолженность)

**Создадим новую таблицу. Основа для таблицы столбы 'children' и 'debt'. \
Третий столбец будет использован для подсчета количества строк в сводной таблице**

*создадим функцию, которая будет возвращать 0, елси детей нет и 1 - если есть*

In [45]:
def having_children_0_1(row):    
    if row['children'] == 0:
        return 0
    else:
        return 1

*создадим новый столбец 'having_children', в котором будут данные о наличие детей. где 0 - нет детей, 1 - есть ребенок/дети

In [46]:
customer_solvency['having_children'] = customer_solvency.apply(having_children_0_1, axis=1)

In [47]:
having_children = customer_solvency[['having_children','debt', 'purpose_loan']]
having_children.head()

,having_children,debt,purpose_loan
0,1,0,кв. м.
1,1,0,автомобиль
2,0,0,кв. м.
3,1,0,образование
4,0,0,свадьба


*На основе нее сделаем сводную таблицу*

In [48]:
having_children_pivot = having_children.pivot_table(index='having_children', columns='debt', values='purpose_loan', aggfunc='count')
having_children_pivot

debt,0,1
having_children,,
0,13026,1063
1,6685,676


#### для задачи №3 (Семейное положение - задолженность)

**Создадим новую таблицу. Основа для таблицы столбы 'family_status' и 'debt'. \
Третий столбец будет использован для подсчета количества строк в сводной таблице**

In [49]:
family_status_debt = customer_solvency[['family_status','debt', 'purpose_loan']]
family_status_debt.head()

,family_status,debt,purpose_loan
0,женат / замужем,0,кв. м.
1,женат / замужем,0,автомобиль
2,женат / замужем,0,кв. м.
3,женат / замужем,0,образование
4,гражданский брак,0,свадьба


*На основе нее сделаем сводную таблицу*

In [50]:
family_status_debt_pivot = family_status_debt.pivot_table(index='family_status', columns='debt', values='purpose_loan', aggfunc='count')
family_status_debt_pivot

debt,0,1
family_status,,
Не женат / не замужем,2535,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3762,388
женат / замужем,11408,929


#### для задачи №4 (Цели кредита - задолженность)

**Создадим новую таблицу. Основа для таблицы столбы 'purpose_loan' и 'debt'. \
Третий столбец будет использован для подсчета количества строк в сводной таблице**

In [51]:
purpose_Loan_debt = customer_solvency[['purpose_loan', 'debt', 'income_type']]
purpose_Loan_debt.head()

,purpose_loan,debt,income_type
0,кв. м.,0,сотрудник
1,автомобиль,0,сотрудник
2,кв. м.,0,сотрудник
3,образование,0,сотрудник
4,свадьба,0,пенсионер


In [52]:
purpose_Loan_debt_pivot = purpose_Loan_debt.pivot_table(index='purpose_loan', columns='debt', values='income_type', aggfunc='count')
purpose_Loan_debt_pivot.sort_values(by=0, ascending=False)

debt,0,1
purpose_loan,,
кв. м.,10027,781
автомобиль,3903,402
образование,3643,370
свадьба,2138,186


#### для задачи №5 (Уровень дохода - задолженность)

**Категоризируем уровни дохода на 'низкий' ( до 35000), 'средний' (от 36000 до 80000), 'высокий' (от 80000)**

In [53]:
def income_level(income):
    if income <= 35000:
        return 'низкий'
    elif 36000 <= income <= 80000:
        return 'средний'
    return 'высокий'

*Проверим функцию*

In [54]:
income_level(5000)

'низкий'

In [55]:
income_level(50000)

'средний'

In [56]:
income_level(90000)

'высокий'

*Создадим новый столбец с категориями заработка*

In [57]:
customer_solvency['income_level'] = customer_solvency['total_income'].apply(income_level)
customer_solvency.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_loan,having_children,income_level
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,кв. м.,1,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,высокий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,кв. м.,0,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,высокий


In [58]:
customer_solvency['income_level'].value_counts()

высокий    19182
средний     2199
низкий        69
Name: income_level, dtype: int64

*Большинство клиентов с высоким заработком*

**Создадим новую таблицу. Основа для таблицы столбы 'income_level' и 'debt'.
Третий столбец будет использован для подсчета количества строк в сводной таблице**

In [59]:
income_debt = customer_solvency[['income_level','debt', 'purpose_loan']]
income_debt.head()

,income_level,debt,purpose_loan
0,высокий,0,кв. м.
1,высокий,0,автомобиль
2,высокий,0,кв. м.
3,высокий,0,образование
4,высокий,0,свадьба


*На основе нее сделаем сводную таблицу*

In [60]:
income_debt_pivot = income_debt.pivot_table(index='income_level', columns='debt', values='purpose_loan', aggfunc='count')
income_debt_pivot.sort_values(by=0, ascending=False)

debt,0,1
income_level,,
высокий,17614,1568
средний,2031,168
низкий,66,3


#### **Вывод**

<div style="border:solid indigo 2px; padding: 20px">

Категоризацию данных выполнили согласно поставленным задачам.
 1. влияние наличия детей на возврат
     - сделали сводную таблицу из столбцов 'children' и 'debt'
 2. влияние семейного положения на возврат
     - сделали сводную таблицу из столбцов 'family_status' и 'debt'
 3. влияние целей кредита на возврат
     - сделали сводную таблицу из столбцов 'purpose_loan' и 'debt'
 4. влияние уровня дохода на возврат
     - категоризировали столбец 'total_income', добавив новый столбец 'income_level' с уровнями заработка: 'низкий' ( до 35000), 'средний' (от 36000 до 80000), 'высокий' (от 80000).
     - сделали сводную таблицу из столбцов 'income_level' и 'debt'

## Шаг 3. Отвечаем на вопросы

### Вопрос 1.  Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [61]:
children_debt_pivot

debt,0,1
children,,
0,13026.0,1063.0
1,4410.0,444.0
2,1926.0,201.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


*Заменим один пропуск на 0, тк у клиентов с 5ю детьми задолженностей нет*

In [62]:
children_debt_pivot[1] = children_debt_pivot[1].fillna(0)
children_debt_pivot

debt,0,1
children,,
0,13026.0,1063.0
1,4410.0,444.0
2,1926.0,201.0
3,303.0,27.0
4,37.0,4.0
5,9.0,0.0


In [63]:
total_count_debt = children_debt_pivot.sum().sum()
total_count_debt 

21450.0

*Напишем функции для вычисления процентных соотношений*

In [64]:
def procent_for_debt_0(row):    # для каждого статуса и задолженности отдельно
    procent_for_debt_0 = row[0] / (row[0] + row[1])*100
    return round(procent_for_debt_0, 2)

def procent_for_debt_1(row):    # для каждого статуса и задолженности отдельно
    procent_for_debt_1 = row[1] / (row[0] + row[1])*100
    return round(procent_for_debt_1, 2)

def procent_total_debt_0(row):    # для процентов от общего числа 
    total = row[[0,1]].sum().sum()
    procent_total_debt_0 = row[0] / total_count_debt *100
    return round(procent_total_debt_0, 2)

def procent_total_debt_1(row):    # для процентов от общего числа 
    procent_total_debt_1 = row[1] / total_count_debt *100
    return round(procent_total_debt_1, 2)

def conversion(row):              # для конверсии
    conversion = round(row['procent_total_debt_1'] / row['procent_total_debt_0'], 2)
    return conversion
    

*Создадим новые колонки с процентными соотношениями и итоговым столбцом конверсии*

In [65]:
children_debt_pivot['procent_for_debt_0'] = children_debt_pivot.apply(procent_for_debt_0, axis=1)
children_debt_pivot['procent_for_debt_1'] = children_debt_pivot.apply(procent_for_debt_1, axis=1)

children_debt_pivot['procent_total_debt_0'] = children_debt_pivot.apply(procent_total_debt_0, axis=1)
children_debt_pivot['procent_total_debt_1'] = children_debt_pivot.apply(procent_total_debt_1, axis=1)

children_debt_pivot['conversion_total'] = children_debt_pivot.apply(conversion, axis=1)

children_debt_pivot.sort_values(by='conversion_total', ascending=True)

debt,0,1,procent_for_debt_0,procent_for_debt_1,procent_total_debt_0,procent_total_debt_1,conversion_total
children,,,,,,,
5,9.0,0.0,100.00,0.00,0.04,0.00,0.00
0,13026.0,1063.0,92.46,7.54,60.73,4.96,0.08
3,303.0,27.0,91.82,8.18,1.41,0.13,0.09
1,4410.0,444.0,90.85,9.15,20.56,2.07,0.10
2,1926.0,201.0,90.55,9.45,8.98,0.94,0.10
4,37.0,4.0,90.24,9.76,0.17,0.02,0.12


#### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">
    
**Рейтинг безопасности:**
    
    1.(Самым безопасным клиент) с 5ю детьми. Просрочено 0,0%.  Видимо, чтобы выдать кредит такому клиенту, банк подумает 100 раз. Банк выдал меньше 1% от общего количества кредитов.
    2. клиент без детей. Просрочено 7,5%. Но банк очень охотно дает кредит - 60% от общего количества.
    3. с 3мя детьми. Просрочено 8%
    4. с 1м ребенком. Просрочено 9,1%
    5. с 2мя детьми. Просрочено 9,5%
    6. с 4мя детьми. Просрочено 9,7% 
    
**Рейтинг количества:**    
    
    1. клиент без детей. Выдано 60% кредитов
    2. с 1м ребенком.    Выдано 20% кредитов
    3. с 2мя детьми.     Выдано 9% кредитов  
    4. с 3мя детьми.     Выдано 1,4% кредитов     
    5. с 4мя детьми.     Выдано 0,2% кредитов    
    5. с 5ми детьми.     Выдано 0,04% кредитов
    
Банк выдает кредиты опираясь на количество детей у клиента. **Чем меньше детей. тем больше кредитов.** \
**Но зависимость между наличием детей и возвратом кредита в срок не выявлена**. Так, самым безопасным клиентом оказался клиент с 5ю детьми. А на 2м месте клиент без детей.
    

    

### Вопрос 2. Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [66]:
having_children_pivot.sort_values(by=0, ascending=False)

debt,0,1
having_children,,
0,13026,1063
1,6685,676


*Используем написанные раннее функции для вычисления процентных соотношений и добавим новые столбцы*

In [67]:
having_children_pivot['procent_for_debt_0'] = having_children_pivot.apply(procent_for_debt_0, axis=1)
having_children_pivot['procent_for_debt_1'] = having_children_pivot.apply(procent_for_debt_1, axis=1)

having_children_pivot['procent_total_debt_0'] = having_children_pivot.apply(procent_total_debt_0, axis=1)
having_children_pivot['procent_total_debt_1'] = having_children_pivot.apply(procent_total_debt_1, axis=1)

having_children_pivot['conversion_total'] = having_children_pivot.apply(conversion, axis=1)

having_children_pivot.sort_values(by='conversion_total', ascending=True)

debt,0,1,procent_for_debt_0,procent_for_debt_1,procent_total_debt_0,procent_total_debt_1,conversion_total
having_children,,,,,,,
0,13026,1063,92.46,7.54,60.73,4.96,0.08
1,6685,676,90.82,9.18,31.17,3.15,0.10


#### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">
    
**Согласно итоговой конверсии 'conversion_total' клиенты без детей возвращают кредиты в срок чаще, чем клиенты с детьми.** \
Клиенты без детей более надежные

### Вопрос 3. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [68]:
family_status_debt_pivot.sort_values(by=0, ascending=False)

debt,0,1
family_status,,
женат / замужем,11408,929
гражданский брак,3762,388
Не женат / не замужем,2535,274
в разводе,1110,85
вдовец / вдова,896,63


In [69]:
family_status_debt_pivot

debt,0,1
family_status,,
Не женат / не замужем,2535,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3762,388
женат / замужем,11408,929


*Используем написанные раннее функции для вычисления процентных соотношений и добавим новые столбцы*

*Создадим новые колонки с процентными соотношениями и итоговым столбцом конверсии*

In [70]:
family_status_debt_pivot['procent_for_debt_0'] = family_status_debt_pivot.apply(procent_for_debt_0, axis=1)
family_status_debt_pivot['procent_for_debt_1'] = family_status_debt_pivot.apply(procent_for_debt_1, axis=1)

family_status_debt_pivot['procent_total_debt_0'] = family_status_debt_pivot.apply(procent_total_debt_0, axis=1)
family_status_debt_pivot['procent_total_debt_1'] = family_status_debt_pivot.apply(procent_total_debt_1, axis=1)

family_status_debt_pivot['conversion_total'] = family_status_debt_pivot.apply(conversion, axis=1)

family_status_debt_pivot.sort_values(by='conversion_total', ascending=True)

debt,0,1,procent_for_debt_0,procent_for_debt_1,procent_total_debt_0,procent_total_debt_1,conversion_total
family_status,,,,,,,
вдовец / вдова,896,63,93.43,6.57,4.18,0.29,0.07
в разводе,1110,85,92.89,7.11,5.17,0.40,0.08
женат / замужем,11408,929,92.47,7.53,53.18,4.33,0.08
гражданский брак,3762,388,90.65,9.35,17.54,1.81,0.10
Не женат / не замужем,2535,274,90.25,9.75,11.82,1.28,0.11


#### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">

**Рейтинг выдачи кредитов**
   - 1 место - женат / замужем.       Выдано 53% от всех кредитов. 
   - 2 место - гражданский брак.      Выдано 18% от всех кредитов. 	
   - 3 место - Не женат / не замужем. Выдано 12% от всех кредитов. 
   - 4 место - в разводе.             Выдано 5% от всех кредитов.     
   - 5 место - вдовец / вдова.        Выдано 4% от всех кредитов.  
    
 **Рейтинг просрочки кредитов**  
   - 1 место - вдовец / вдова.        Просрочено 0,3%.
   - 2 место - в разводе.             Просрочено 0,4%.
   - 3 место - Не женат / не замужем. Просрочено 1,2%.
   - 4 место - гражданский брак.      Просрочено 1,8%.    
   - 5 место - женат / замужем.       Просрочено 4,3%.
 
 **Рейтинг конверсии просроченных кредитов (от лучшего к худшему)** 
   - 1 место - вдовец / вдова.         Конверсия 0,07%.
   -  2 место - в разводе.             Конверсия 0,08%.
   -  3 место - женат / замужем.       Конверсия 0.08%.
   -  4 место - гражданский брак.      Конверсия 0.10%.    
   -  5 место - Не женат / не замужем. Конверсия 0.11%. 
 
 
 Клиентам со статусом 'женат / замужем' выдают кредиты охотнее остальных.  Конверсия просроченности данной группы на 3м месте, средняя. В данной категории рекомундую оставить количество выданных кредитов на том же уровне
 
 Лучше всего с задолженностью в категории 'вдовец / вдова'. В данной категории рекомундую выдавать больше кредитов. \
 Хуже всего с задолженностью в категории 'Не женат / не замужем'. В данной категории рекомундую выдавать меньше кредитов

**Зависимость между семейным положением и возвратом кредита в срок присутствует.**
Клиенты со статусом женат/замужем или в разводе возвращают кредит вовремя чаще, чем не женатые/ не замужем.


### Вопрос 4. Как разные цели кредита влияют на его возврат в срок?

In [71]:
purpose_Loan_debt_pivot.sort_values(by=0, ascending=False)

debt,0,1
purpose_loan,,
кв. м.,10027,781
автомобиль,3903,402
образование,3643,370
свадьба,2138,186


*Используем написанные раннее функции для вычисления процентных соотношений и добавим новые столбцы*

In [72]:
purpose_Loan_debt_pivot['procent_for_debt_0'] = purpose_Loan_debt_pivot.apply(procent_for_debt_0, axis=1)
purpose_Loan_debt_pivot['procent_for_debt_1'] = purpose_Loan_debt_pivot.apply(procent_for_debt_1, axis=1)

purpose_Loan_debt_pivot['procent_total_debt_0'] = purpose_Loan_debt_pivot.apply(procent_total_debt_0, axis=1)
purpose_Loan_debt_pivot['procent_total_debt_1'] = purpose_Loan_debt_pivot.apply(procent_total_debt_1, axis=1)

purpose_Loan_debt_pivot['conversion_total'] = purpose_Loan_debt_pivot.apply(conversion, axis=1)

purpose_Loan_debt_pivot.sort_values(by='conversion_total', ascending=True)

debt,0,1,procent_for_debt_0,procent_for_debt_1,procent_total_debt_0,procent_total_debt_1,conversion_total
purpose_loan,,,,,,,
кв. м.,10027,781,92.77,7.23,46.75,3.64,0.08
свадьба,2138,186,92.00,8.00,9.97,0.87,0.09
автомобиль,3903,402,90.66,9.34,18.20,1.87,0.10
образование,3643,370,90.78,9.22,16.98,1.72,0.10


#### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">

- По результатам итоговой конверсии цель кредита 'кв м' (на разную недвижимость) имеет минимальную степень возврата.
    А как иначе, банк ведь заберет недвижимость! 
- Хуже всего дела обстоят с кредитами на автомобиль и образование.
    Образование в случае невозврата кредита можно перенести, автомобиль видимо является не такой ценностью, как недвижимость, можно пожертвовать. Отсюда и вытекают просрочки кредита.

### Вопрос 5. Как уровень дохода влияет на возврат кредита в срок?

In [73]:
income_debt_pivot.sort_values(by=0, ascending=False)

debt,0,1
income_level,,
высокий,17614,1568
средний,2031,168
низкий,66,3


*Используем написанные раннее функции для вычисления процентных соотношений и добавим новые столбцы*

In [74]:
income_debt_pivot['procent_for_debt_0'] = income_debt_pivot.apply(procent_for_debt_0, axis=1)
income_debt_pivot['procent_for_debt_1'] = income_debt_pivot.apply(procent_for_debt_1, axis=1)

income_debt_pivot['procent_total_debt_0'] = income_debt_pivot.apply(procent_total_debt_0, axis=1)
income_debt_pivot['procent_total_debt_1'] = income_debt_pivot.apply(procent_total_debt_1, axis=1)

income_debt_pivot['conversion_total'] = income_debt_pivot.apply(conversion, axis=1)

income_debt_pivot.sort_values(by='conversion_total', ascending=True)

debt,0,1,procent_for_debt_0,procent_for_debt_1,procent_total_debt_0,procent_total_debt_1,conversion_total
income_level,,,,,,,
низкий,66,3,95.65,4.35,0.31,0.01,0.03
средний,2031,168,92.36,7.64,9.47,0.78,0.08
высокий,17614,1568,91.83,8.17,82.12,7.31,0.09


#### **Вывод**

 <div style="border:solid indigo 2px; padding: 20px">

- По результатам итоговой конверсии клиенты с низким уровнем дохода имеют минимальную степень возврата.
    Отдают кредит вовремя, тк могут потерять больше. \
- Хуже всего дела обстоят с клиентами с высоким уровнем дохода.
    Таких клиентов не беспокоят возможные санкции. Могут себе позволить задержать кредит при необхождимости, оплатив пеню.
 
 **Чем выше уровень дохода, тем чаще клиенты нарушают требование возврата кредита в срок.**

## Шаг 4. Общий вывод

 <div style="border:solid indigo 5px; padding: 20px">

После анализа данных можно сделать выводы о том, что не все аспекты семейного положения влияют на возврат кредита в срок.
    
      - На вопрос: Есть ли зависимость между наличием детей и возвратом кредита в срок? 
 **Закономерность не выявлена**. Так, самым безопасным клиентом оказался клиент с 5ю детьми. А на 2м месте клиент без детей.
    
       - На вопрос: Есть ли зависимость между наличием детей и возвратом кредита в срок?    
 Выяснили, что **клиенты без детей возвращают кредиты в срок чаще, чем клиенты с детьми** 
    Это может быть связано с дополнительными тратами на ребенка:сезонной одеждой, на инвентарь и тд.
    
      - На вопрос: Есть ли зависимость между семейным положением и возвратом кредита в срок?
 **Закономерность выявлена**. Клиенты со статусом 'женат/замужем' или 'в разводе' возвращают кредит в установленный срок чаще, чем не женатые/ не замужем.
  
     -  На вопрос: Как разные цели кредита влияют на его возврат в срок?
 Выяснили, что **цель кредита 'кв м' (на разную недвижимость) имеет минимальную степень задержки возврата.** А как иначе, банк ведь заберет недвижимость!
**Хуже всего дела обстоят с кредитами на автомобиль и образование.** Образование в случае невозврата кредита можно перенести, автомобиль видимо является не такой ценностью, как недвижимость, можно пожертвовать. Отсюда и вытекают просрочки кредита
    
     -  На вопрос: Как уровень дохода влияет на возврат кредита в срок?
Выяснили, что **чем выше уровень дохода, тем чаще клиенты нарушают требование возврата кредита в срок.** 
    Клиенты с высоким доходом могут себе позволить задержать кредит при необхождимости, оплатив пеню.
    Клиенты с низким уровнем дохода стараются всегда возвращать кредит в срок, чтобы не тратить лишние деньги.